<a href="https://colab.research.google.com/github/prado3003/virtual_assitant/blob/main/virtual_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gTTS SpeechRecognition pydub
!apt-get install -y ffmpeg


In [ ]:
import re
from gtts import gTTS
from google.colab import output
from IPython.display import Audio, display, HTML
from base64 import b64decode
from pydub import AudioSegment
import speech_recognition as sr

def assistente_colab(duration=5):
    # injeta JS p/ capturar áudio no navegador
    to_js = f"""
    async function record(duration) {{
      const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
      const recorder = new MediaRecorder(stream);
      let data = [];
      recorder.ondataavailable = e => data.push(e.data);
      recorder.start();
      await new Promise(r => setTimeout(r, duration * 1000));
      recorder.stop();
      await new Promise(r => recorder.onstop = r);
      let blob = new Blob(data, {{ type: 'audio/webm' }});
      let arrayBuffer = await blob.arrayBuffer();
      let base64 = btoa(String.fromCharCode(...new Uint8Array(arrayBuffer)));
      return base64;
    }}
    record({duration});
    """
    print(f"🎤 Gravando {duration} segundos... fale agora!")
    audio_bytes = output.eval_js(to_js)

    # converte WebM -> WAV válido
    with open("temp.webm", "wb") as f:
        f.write(b64decode(audio_bytes))
    audio = AudioSegment.from_file("temp.webm", format="webm")
    audio = audio.set_channels(1).set_frame_rate(16000)
    audio.export("input.wav", format="wav")

    # transcreve áudio
    recognizer = sr.Recognizer()
    with sr.AudioFile("input.wav") as source:
        audio_data = recognizer.record(source)
        try:
            texto = recognizer.recognize_google(audio_data, language="pt-BR")
            print("📝 Você disse:", texto)
        except sr.UnknownValueError:
            texto = ""
            print("😕 Não entendi o que você falou.")
        except sr.RequestError as e:
            texto = ""
            print("❌ Erro no serviço de reconhecimento:", e)

    # resposta baseada no comando
    link = None
    texto_lower = texto.lower()
    if "wikipedia" in texto_lower:
        resposta = "Ok, vou abrir o Wikipedia para você."
        link = "https://pt.wikipedia.org"
    elif "youtube" in texto_lower:
        resposta = "Beleza, abrindo o YouTube agora."
        link = "https://www.youtube.com"
    elif "farmácia" in texto_lower:
        resposta = "Aqui está a farmácia mais próxima que encontrei no Google Maps."
        link = "https://www.google.com/maps/search/farmácia+perto+de+mim"
    else:
        resposta = "Comando não reconhecido."

    # exibe resposta e link clicável (se existir)
    if link:
        display(HTML(f'<a href="{link}" target="_blank">🔗 Abrir link</a>'))

    # fala resposta
    tts = gTTS(resposta, lang="pt")
    tts.save("resposta.mp3")
    display(Audio("resposta.mp3", autoplay=True))


In [ ]:
assistente_colab(5)  # grava 5 segundos
